#  Projet FcMetz — Présentation

**Objectif du projet :** Analyser les performances des joueurs et équipes de Ligue 1 en combinant trois sources de données :
-  **StatsBomb** — Événements détaillés (passes, tirs, xG, dribbles...)
-  **SkillCorner** — Données physiques (vitesse, sprints, distance parcourue...)
-  **Transfermarkt** — Valeurs marchandes des joueurs (via webscraping)

---

## 1. Architecture du projet

Le projet est organisé en modules Python séparés pour bien distinguer chaque source de données :

```
FcMetz_Data/
│
├── config.py                    # Identifiants & paramètres centraux
├── pipeline.py                  # Pipeline d'ingestion DuckDB (partiel — voir README)
│
├── statsbomb/
│   ├── connexion.py             # Authentification HTTP à l'API
│   ├── competitions.py          # Récupération des compétitions / saisons
│   ├── matchs.py                # Récupération des matchs
│   └── evenements.py            # Récupération des événements de jeu
│
├── skillcorner/
│   └── tracking.py              # Données physiques de tracking
│
├── transfermarkt/
│   └── scraping.py              # Webscraping des valeurs marchandes
│
├── donnees/
│   └── football.duckdb          # Base de données DuckDB (générée par le pipeline)
│
└── notebook.ipynb               # Ce notebook — exploration et tests des sources
```

Chaque module est indépendant. On peut utiliser StatsBomb seul, SkillCorner seul, ou les combiner.  
Le fichier `pipeline.py` centralise l'ingestion vers DuckDB (voir README pour le détail).

---
##  2. API StatsBomb

### Comment ça fonctionne ?

StatsBomb expose ses données via une **API REST** : on envoie une requête HTTP avec nos identifiants, et on reçoit les données au format JSON.

**Hiérarchie des données :**

 Competitions,
 Saisons,
 Matchs,
 Événements,


**URL principales utilisées :**

 Compétitions :'GET/api/v4/competitions` 

 Matchs :`GET /api/v6/competitions/{id}/seasons/{id}/matches` 
 
 Événements:`GET /api/v8/events/{match_id}` 

*Import*

In [1]:
#  Imports 
import sys
sys.path.append('..') 

from statsbomb.competitions import recuperer_saisons_ligue1, recuperer_saison_recente_ligue1
from statsbomb.matchs import recuperer_matchs
from statsbomb.evenements import recuperer_evenements, filtrer_par_type
import pandas as pd

### 2.1 — Les saisons Ligue 1 disponibles

In [2]:
# On récupère toutes les saisons Ligue 1 disponibles dans la base de données StatsBomb
saisons = recuperer_saisons_ligue1()

print(f"Nombre de saisons disponibles : {len(saisons)}\n")
saisons[['competition_id', 'competition_name', 'season_id', 'season_name']]

Nombre de saisons disponibles : 2



,competition_id,competition_name,season_id,season_name
0,7,Ligue 1,318,2025/2026
1,7,Ligue 1,317,2024/2025


### 2.2 — Les matchs de la saison la plus récente

In [3]:
# On récupère automatiquement la saison la plus récente
saison = recuperer_saison_recente_ligue1()
print(f"Saison sélectionnée : {saison['season_name']}")

# On charge les matchs
matchs = recuperer_matchs(saison['competition_id'], saison['season_id'])

print(f"\nNombre de matchs : {len(matchs)}")
matchs.head(10)

Saison sélectionnée : 2025/2026

Nombre de matchs : 306


,match_id,match_date,kick_off,equipe_domicile,home_score,equipe_exterieur,away_score,stade,arbitre,match_week
0,3999401,2025-08-15,18:45:00.000,Rennes,1.0,Marseille,0.0,Roazhon Park,Jérémie Pignard,1
1,3999403,2025-08-16,15:00:00.000,Lens,0.0,Lyon,1.0,Stade Bollaert-Delelis,Romain Lissorgue,1
2,3999404,2025-08-16,19:05:00.000,OGC Nice,0.0,Toulouse,1.0,Allianz Riviera,Thomas Léonard,1
3,3999407,2025-08-16,17:00:00.000,AS Monaco,3.0,Le Havre,1.0,Stade Louis II,Ruddy Buquet,1
4,3999402,2025-08-17,13:00:00.000,Stade Brestois,3.0,Lille,3.0,Stade Francis-Le Blé,Clément Turpin,1
5,3999405,2025-08-17,18:45:00.000,Nantes,0.0,Paris Saint-Germain,1.0,Stade de la Beaujoire - Louis Fonteneau,Benoît Bastien,1
6,3999399,2025-08-17,15:15:00.000,Angers,1.0,Paris FC,0.0,Stade Raymond-Kopa,François Letexier,1
7,3999406,2025-08-17,15:15:00.000,Metz,0.0,Strasbourg,1.0,Stade Saint-Symphorien,Mathieu Vernice,1
8,3999400,2025-08-17,15:15:00.000,Auxerre,1.0,Lorient,0.0,Stade de l'Abbé Deschamps,Abdelatif Kherradji,1
9,3999416,2025-08-22,18:45:00.000,Paris Saint-Germain,1.0,Angers,0.0,Parc des Princes,Hakim Ben El Hadj Salem,2


### 2.3 — Les événements d'un match

In [4]:
# On prend le premier match de la saison
premier_match = matchs.iloc[0]
nom_match = f"{premier_match['equipe_domicile']} vs {premier_match['equipe_exterieur']}"

print(f"Match analysé : {nom_match} ({premier_match['match_date']})")

# Chargement de tous les événements
evenements = recuperer_evenements(premier_match['match_id'])

print(f"\nNombre total d'événements : {len(evenements)}\n")
print("Répartition par type :")
print(evenements['type_evenement'].value_counts())

Match analysé : Rennes vs Marseille (2025-08-15)

Nombre total d'événements : 3729

Répartition par type :
type_evenement
Pass               1104
Ball Receipt*      1000
Carry               863
Pressure            336
Ball Recovery        76
Block                51
Goal Keeper          44
Duel                 41
Shot                 36
Clearance            35
Foul Committed       22
Foul Won             21
Dribble              18
Miscontrol           15
Dispossessed         13
Interception         10
Substitution         10
Dribbled Past         7
Tactical Shift        5
Half Start            4
Injury Stoppage       4
Half End              4
Starting XI           2
Bad Behaviour         2
50/50                 2
Error                 1
Player Off            1
Player On             1
Shield                1
Name: count, dtype: int64


In [5]:
# extraire uniquement les tirs du match
tirs = filtrer_par_type(evenements, 'Shot')

print(f"Nombre de tirs : {len(tirs)}\n")
tirs[['joueur', 'equipe', 'minute', 'type_evenement']].head(10)

Nombre de tirs : 36



,joueur,equipe,minute,type_evenement
398,Amine Gouiri,Marseille,8,Shot
619,Mason Greenwood,Marseille,12,Shot
784,Moussa Al Tamari,Rennes,16,Shot
814,Amine Gouiri,Marseille,17,Shot
843,Moussa Al Tamari,Rennes,19,Shot
1288,Mason Greenwood,Marseille,37,Shot
1308,Jonathan David Henry Rowe,Marseille,38,Shot
1321,Pierre-Emile Højbjerg,Marseille,38,Shot
1406,Mason Greenwood,Marseille,40,Shot
1439,Moussa Al Tamari,Rennes,41,Shot


---
##  3. API SkillCorner — Données physiques

### Comment ça fonctionne ?

SkillCorner utilise la **vision par ordinateur** (analyse vidéo) pour tracker les positions et mouvements de chaque joueur sur le terrain.

**Ce qu'on peut récupérer :**
-  Distance totale parcourue (mètres)
-  Nombre de sprints (> 25 km/h)
-  Vitesse maximale atteinte
-  Courses à haute intensité (entre 19.8 et 25 km/h)
-  Courses sans ballon (*off-ball runs*)

In [6]:
from skillcorner.tracking import recuperer_matchs_disponibles, recuperer_tracking_match

# Liste des matchs disponibles sur le compte SkillCorner
matchs_sk = recuperer_matchs_disponibles()

print(f"Matchs disponibles sur SkillCorner : {len(matchs_sk)}")
matchs_sk.head(5)

Matchs disponibles sur SkillCorner : 414


,id,date_time,home_team,away_team,status,competition_id,season_id,competition_edition_id
0,2055232,2026-02-21T20:05:00Z,"{'id': 76, 'short_name': 'Paris'}","{'id': 97, 'short_name': 'FC Metz'}",closed,3,129,1232
1,2055352,2026-02-21T19:00:00Z,"{'id': 84, 'short_name': 'Saint-Étienne'}","{'id': 735, 'short_name': 'Stade Lavallois'}",closed,12,129,1248
2,2055169,2026-02-21T18:00:00Z,"{'id': 74, 'short_name': 'Toulouse FC'}","{'id': 327, 'short_name': 'Paris FC'}",closed,3,129,1232
3,2055170,2026-02-21T16:00:00Z,"{'id': 85, 'short_name': 'RC Lens'}","{'id': 75, 'short_name': 'AS Monaco'}",closed,3,129,1232
4,2055146,2026-02-21T13:00:00Z,"{'id': 95, 'short_name': 'Troyes AC'}","{'id': 734, 'short_name': 'Pau FC'}",closed,12,129,1248


In [7]:
# Données physiques du premier match
id_match_sk = matchs_sk.iloc[0]['id']
tracking = recuperer_tracking_match(id_match_sk)

print("Données physiques par joueur :")
tracking.head(10)

Données physiques par joueur :


,player_name,player_short_name,player_id,player_birthdate,team_name,team_id,match_name,match_id,match_date,competition_name,...,highdecel_count_full_all,explacceltohsr_count_full_all,timetohsr,timetohsrpostcod,explacceltosprint_count_full_all,timetosprint,timetosprintpostcod,cod_count_full_all,timeto505around90,timeto505around180
0,Bouna Sarr,B. Sarr,1420,1992-01-31,FC Metz,97,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,8.0,1.0,0.81,0.81,0.0,NaN,NaN,19.0,2.37,2.98
1,Jean-Philippe Gbamin,J. Gbamin,4865,1995-09-25,FC Metz,97,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,11.0,3.0,0.93,0.93,0.0,NaN,NaN,43.0,2.19,2.93
2,Lucas François Bernard Hernández Pi,L. Hernández,6357,1996-02-14,Paris Saint-Germain,76,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,19.0,5.0,0.74,0.74,3.0,1.40,1.63,40.0,2.13,2.85
3,Fodé Ballo-Touré,F. Ballo-Touré,11450,1997-01-03,FC Metz,97,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,15.0,4.0,0.60,NaN,4.0,1.40,NaN,26.0,2.85,2.78
4,Achraf Hakimi Mouh,A. Hakimi,11495,1998-11-04,Paris Saint-Germain,76,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,24.0,7.0,0.78,0.80,4.0,1.34,1.45,50.0,2.38,3.01
5,Jessy Deminguet,J. Deminguet,11504,1998-01-07,FC Metz,97,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,4.0,1.0,0.84,NaN,0.0,NaN,NaN,18.0,2.68,2.49
6,Georgiy Tsitaishvili,G. Tsitaishvili,12381,2000-11-18,FC Metz,97,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,30.0,5.0,0.76,0.79,2.0,1.40,1.40,47.0,2.29,3.16
7,Giorgi Kvilitaia,G. Kvilitaia,12472,1993-10-01,FC Metz,97,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,2.0,2.0,0.78,0.78,1.0,1.80,NaN,12.0,2.64,2.83
8,Habib Mouhamadou Diallo,H. Diallo,13005,1995-06-18,FC Metz,97,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,10.0,5.0,0.79,0.88,0.0,NaN,NaN,35.0,2.68,2.94
9,Kang-In Lee,Lee Kang-In,20236,2001-02-19,Paris Saint-Germain,76,Paris Saint-Germain v FC Metz,2055232,2026-02-21,FRA - Ligue 1,...,9.0,2.0,0.76,0.90,0.0,NaN,NaN,34.0,2.49,2.99


---
##  4. Transfermarkt — Webscraping des valeurs marchandes

### Comment ça fonctionne ?

Contrairement à StatsBomb et SkillCorner, Transfermarkt **n'a pas d'API officielle**. On utilise donc le **webscraping** : on simule un navigateur pour lire le HTML de la page et en extraire les données.

**Outil utilisé : `BeautifulSoup`** — une bibliothèque Python qui parse le HTML.

- Délai entre chaque requête pour ne pas surcharger le site
- En-tête `User-Agent` pour simuler un navigateur réel

In [1]:
from transfermarkt.scraping import recuperer_equipes_ligue1, recuperer_joueurs_equipe

# On récupère d'abord la liste des équipes de Ligue 1
equipes = recuperer_equipes_ligue1()

print(f"Équipes trouvées : {len(equipes)}")
for eq in equipes[:5]:
    print(f" - {eq['equipe']}")

Équipes trouvées : 18
 - Paris Saint-Germain
 - Olympique de Marseille
 - AS Monaco
 - RC Strasbourg Alsace
 - LOSC Lille


In [2]:
# On scrape les joueurs d'une équipe (exemple : la première de la liste)
premiere_equipe = equipes[0]
print(f"Scraping de : {premiere_equipe['equipe']}...")

joueurs = recuperer_joueurs_equipe(premiere_equipe['url_transfermarkt'])

print(f"\nNombre de joueurs : {len(joueurs)}")
joueurs.head(10)

Scraping de : Paris Saint-Germain...

Nombre de joueurs : 24


,joueur,valeur_marchande
0,Lucas Chevalier,"35,00 mio. €"
1,Matvey Safonov,"15,00 mio. €"
2,Renato Marin,500 K €
3,Willian Pacho,"70,00 mio. €"
4,Ilya Zabarnyi,"50,00 mio. €"
5,Marquinhos,"30,00 mio. €"
6,Lucas Beraldo,"20,00 mio. €"
7,Nuno Mendes,"75,00 mio. €"
8,Lucas Hernández,"20,00 mio. €"
9,Achraf Hakimi,"80,00 mio. €"


---
##  5. Combinaison des trois sources

L'intérêt principal du projet est de **croiser** ces données pour des analyses :

 Question d'analyse ?  

 Un joueur cher crée-t-il plus d'occasions ?  StatsBomb (xG) + Transfermarkt (valeur) 

 Les joueurs qui sprintent le plus tirent-ils plus ?  SkillCorner (sprints) + StatsBomb (tirs) 
 
 Quel rapport performance / valeur marchande ?  StatsBomb + SkillCorner + Transfermarkt 


##  6. Récapitulatif 

 Source | Type d'accès | Authentification | Format |
|---|---|---|---|
| StatsBomb | API REST | Login / Mot de passe | JSON |
| SkillCorner | API REST | Login / Mot de passe | JSON |
| Transfermarkt | Webscraping | Aucune | HTML, parse |

**Bibliothèques Python utilisées :**
- `requests` — pour les appels HTTP (API StatsBomb & SkillCorner)
- `pandas` — pour manipuler les tableaux de données
- `beautifulsoup4` — pour parser le HTML de Transfermarkt